In [ ]:
from bidict import bidict
import numpy as np
from time import time
from traceback import format_exc
import codecs
from scipy.sparse import csr_matrix
from math import floor


class CRSGraph(object):
    """ A graph based on the CSR sparse matrix data structure. """

    def __init__(self, neighbors_fpath):
        self._graph, self.index = self._load(neighbors_fpath) 
      
    
    def _get_or_add(self, dictionary, value):
        """ Gets the key associated with the value if exists. 
        Otherwiese inserts the value eq. to the length of the 
        dictionary and returns the key. """

        if value not in dictionary:
            value_idx = len(dictionary)
            dictionary[value] = len(dictionary)
        else:
            value_idx = dictionary[value]

        return value_idx

    
    def _load(self, neighbors_fpath):   
        tic = time()
        with codecs.open(neighbors_fpath, "r", "utf-8") as graph:
            src_lst = []
            dst_lst = []
            data_lst = []
            index = bidict()
            word_dict = {}
            for i, line in enumerate(graph):                
                if i % 10000000 == 0 and i != 0: print(i)
                try:
                    src, dst, weight = line.split("\t")
                    src = src.strip()
                    dst = dst.strip()
                    src_idx = self._get_or_add(index, src)
                    dst_idx = self._get_or_add(index, dst) 

                    src_lst.append(int(src_idx))
                    dst_lst.append(int(dst_idx))
                    data_lst.append(np.int16(floor(float(weight) * 10000.)))
                except:
                    print(format_exc())
                    print("Bad line:", line)

        rows = np.array(src_lst)
        cols = np.array(dst_lst)
        data = np.array(data_lst, dtype=np.int16)
        graph = csr_matrix( (data, (rows, cols)), shape=(len(index),len(index)), dtype=np.int16 )       
        print("Loaded in {:f} sec.".format(time() - tic))

        return graph, index 

    def get_neighbors(self, word):
        idx_i = self.index[word]
        
#         nns = {self.index.inv[idx_j]: self._graph[idx_i].data[j] 
#             for j, idx_j in enumerate(self._graph[idx_i].indices)}
        data_i = self._graph[idx_i].data
        nns = {self.index.inv[idx_j]: data_i[j] 
               for j, idx_j in enumerate(self._graph[idx_i].indices)}
        
        return nns
       
    def get_weight(self, word_i, word_j):
        return 0.0



from itertools import zip_longest
import argparse, sys, subprocess
from utils.common import exists
from os.path import basename
import gensim
import gzip
from gensim.models.keyedvectors import KeyedVectors
from gensim.utils import tokenize
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from time import time
import numpy as np
from chinese_whispers import chinese_whispers, aggregate_clusters
import codecs
import networkx as nx
from multiprocessing import Pool
from os.path import join
from collections import defaultdict
import codecs 
from time import time

import filter_clusters
import vector_representations.build_sense_vectors
from utils.common import ensure_dir
import pcz

verbose = True


def get_ego_network(ego):
    tic = time()
    ego_network = nx.Graph(name=ego)
    
    # Add related and substring nodes
    substring_nodes = []
    for j, node in enumerate(G.index):
        if ego.lower() == node.lower():
            nns_node = G.get_neighbors(node)
            ego_nodes = [(rn, {"weight": w}) for rn, w in nns_node.items()]
            ego_network.add_nodes_from(ego_nodes)
        else:
            if "_" not in node: continue
            if node.startswith(ego + "_") or node.endswith("_" + ego):
                substring_nodes.append( (node, {"weight": 9999}) )
    ego_network.add_nodes_from(substring_nodes)
    
    # Find edges of the ego network
    for r_node in ego_network:
        related_related_nodes = G.get_neighbors(r_node)
        related_related_nodes_ego = sorted(
            [(related_related_nodes[rr_node], rr_node) for rr_node in related_related_nodes if rr_node in ego_network],
            reverse=True)[:n]
        related_edges = [(r_node, rr_node, {"weight": w}) for w, rr_node in  related_related_nodes_ego]
        ego_network.add_edges_from(related_edges)
    
    chinese_whispers(ego_network, weighting="top", iterations=20)
    if verbose: print("{}\t{:f} sec.".format(ego, time()-tic))

    return ego_network


G = None
n = None


def ego_network_clustering(neighbors_fpath, clusters_fpath, max_related=300, num_cores=32):
    global G
    global n
    G = graph 
    
    with codecs.open(clusters_fpath, "w", "utf-8") as output, Pool(num_cores) as pool:
        output.write("word\tcid\tcluster\tisas\n")

        for i, ego_network in enumerate(pool.imap_unordered(get_ego_network, G.index)):
            if i % 100 == 0: print(i, "ego networks processed")
            sense_num = 1
            for label, cluster in sorted(aggregate_clusters(ego_network).items(), key=lambda e: len(e[1]), reverse=True):
                output.write("{}\t{}\t{}\t\n".format(
                    ego_network.name,
                    sense_num,
                    ", ".join( ["{}:{:.4f}".format(c_node, ego_network.node[c_node]["weight"]) for c_node in cluster] )
                ))
                sense_num += 1
    print("Clusters:", clusters_fpath)

       
neighbors_fpath = "model/wiki.txt.graph"
clusters_fpath = "model/tmp-wiki.txt"
graph = CRSGraph(neighbors_fpath)
ego_network_clustering(neighbors_fpath, clusters_fpath, max_related=300, num_cores=12)


Loading spacy model...
10000000
20000000
30000000
40000000
Loaded in 590.774360 sec.
entire	1.183476 sec.
itself	1.205698 sec.
doing_so	1.167330 sec.
another	1.206016 sec.
whole	1.290742 sec.
0 ego networks processed
this	1.642233 sec.
the	1.733156 sec.
of	1.799195 sec.
chaos	1.752683 sec.
its	1.929380 sec.
thus	1.928896 sec.
The	2.062204 sec.
a	0.921822 sec.
each	0.942959 sec.
proper	1.100072 sec.
effectively	1.351096 sec.
ultimately	1.154752 sec.
standing	1.023737 sec.
it	1.726388 sec.
dead	1.377672 sec.
its_own	1.179950 sec.
humanity	1.371479 sec.
original	1.312470 sec.
permanent	1.623927 sec.
opening	1.242080 sec.
immediately	1.129591 sec.
firmly	0.667464 sec.
again	1.774057 sec.
immediate	1.272575 sec.
s	2.048401 sec.
second	1.285403 sec.
their	1.272150 sec.
advancing	1.085087 sec.
final	1.865255 sec.
one	1.846856 sec.
strength	1.326975 sec.
tension	1.178005 sec.
rest	1.142275 sec.
slavery	1.299398 sec.
fixed	1.016760 sec.
once_again	1.359231 sec.
temporary	1.189327 sec.
unity	1.4

when	1.194144 sec.
patrons	0.982203 sec.
Several	1.076716 sec.
obstacles	1.161958 sec.
continually	0.580549 sec.
Often	1.138020 sec.
eyes	1.483747 sec.
Once	1.386515 sec.
deeply	1.046552 sec.
barriers	1.865100 sec.
contacts	1.048539 sec.
Some	1.146101 sec.
although	0.815125 sec.
Those_who	1.514224 sec.
increasingly	1.483363 sec.
In_addition	0.723304 sec.
if_they	0.601777 sec.
who	1.986802 sec.
opportunities	1.316231 sec.
Numerous	1.055932 sec.
excellent	1.160526 sec.
Additionally	1.243288 sec.
equally	1.146169 sec.
Soon	0.990152 sec.
frequently	1.074733 sec.
hands	1.520123 sec.
whom	0.996078 sec.
gifts	0.999826 sec.
helping	1.142133 sec.
because	1.244973 sec.
those_who	1.884896 sec.
But	1.080382 sec.
Their	1.087257 sec.
Both_men	0.632811 sec.
numerous	0.999688 sec.
various	0.968412 sec.
Because	1.016909 sec.
symbolic_meanings	0.562452 sec.
clothes	1.396413 sec.
family_members	0.720406 sec.
townsfolk	0.619058 sec.
harsh	1.050489 sec.
Moreover	1.342750 sec.
neighbors	1.656903 sec.
heavil

actively	0.666396 sec.
let_them	0.618068 sec.
very_difficult	0.678158 sec.
we_can	1.334042 sec.
compelled	0.729443 sec.
anyway	1.086288 sec.
temporarily	0.939362 sec.
fresh	1.422411 sec.
promised	1.049412 sec.
too_far	1.293131 sec.
would_inevitably	0.589177 sec.
we_should	0.732562 sec.
intended	1.661517 sec.
deliberately	0.548967 sec.
further	1.650488 sec.
more_difficult	0.698277 sec.
both_parties	0.665386 sec.
hoping	1.608725 sec.
helps	1.690709 sec.
thereto	0.656085 sec.
ready	1.099653 sec.
an_opportunity	0.594128 sec.
safely	0.554848 sec.
stronger	1.482094 sec.
helped_him	0.686248 sec.
safer	1.143140 sec.
ability	1.160285 sec.
intent	1.199401 sec.
lift	1.226485 sec.
difficult	1.262232 sec.
creditors	1.275697 sec.
reluctance	0.820780 sec.
refusing	1.448067 sec.
Let_us	1.495161 sec.
harder	1.081582 sec.
their_opponents	0.638339 sec.
appropriate	1.197942 sec.
aiming	1.373911 sec.
failed	1.251909 sec.
hard	1.274748 sec.
we_cannot	1.138710 sec.
would_eventually	0.760549 sec.
couldn_t	1.3

exhibits	1.290083 sec.
assumes	0.789846 sec.
quantity	1.274006 sec.
difference	1.269487 sec.
receives	0.627333 sec.
ambiguity	1.196106 sec.
parameter	1.376408 sec.
develops	1.200333 sec.
conveys	0.670217 sec.
contributes	0.659133 sec.
rests_on	0.749941 sec.
takes_place	0.534855 sec.
be	3.171162 sec.
expands	0.596240 sec.
decreases	0.762508 sec.
magnification	1.148766 sec.
locality	1.256409 sec.
variable	1.272665 sec.
rises	1.092974 sec.
surroundings	1.255335 sec.
wasn_t	1.403965 sec.
favors	1.342211 sec.
removes	0.707447 sec.
can_occur	0.602138 sec.
probability_distribution	0.824666 sec.
1100 ego networks processed
knows	1.192053 sec.
ends_up	0.607052 sec.
remains_unclear	0.765595 sec.
relates	0.737332 sec.
eliminates	0.666818 sec.
having_been	0.944957 sec.
there_exists	0.920090 sec.
determinant	0.745865 sec.
specifies	0.585114 sec.
constraint	1.121071 sec.
operates	0.621668 sec.
knot	1.257177 sec.
difference_between	2.854545 sec.
threshold	1.017839 sec.
flows	1.201509 sec.
fraction	1.

savings	1.254140 sec.
fund	1.433973 sec.
job	1.116636 sec.
bad	1.837906 sec.
ample	0.681045 sec.
supporting	1.112203 sec.
expenses	1.122912 sec.
safety	1.352868 sec.
lasting	1.148441 sec.
jobs	1.307132 sec.
customer	1.327876 sec.
cost	1.331557 sec.
tax	1.314223 sec.
insurance	1.330785 sec.
avoiding	1.498158 sec.
incapable	0.604886 sec.
skills	1.464968 sec.
aid	1.276046 sec.
additional	1.629806 sec.
service	1.779411 sec.
fee	1.140162 sec.
income	1.705098 sec.
adequate	1.292924 sec.
practical	1.564128 sec.
medical_care	0.631374 sec.
supply	1.791048 sec.
extremely_costly	0.655863 sec.
ease	1.134743 sec.
cash	1.323731 sec.
consideration	1.323961 sec.
cheap	1.335978 sec.
task	1.064979 sec.
prospective	1.055420 sec.
payments	1.534233 sec.
recognition	1.115287 sec.
suitable_for	0.605919 sec.
maintenance	1.416017 sec.
monetary	1.724143 sec.
delayed	1.231378 sec.
credit_card	0.928873 sec.
labour	1.301720 sec.
flexibility	1.116472 sec.
reliable	1.086028 sec.
check	1.345560 sec.
rewards	2.132394 

attitude	1.102683 sec.
metaphor	1.247066 sec.
acting	1.388154 sec.
excessively	0.567901 sec.
collectively	1.146611 sec.
weak	1.365819 sec.
obviously	1.285015 sec.
fundamentally	1.138856 sec.
non_existent	0.545733 sec.
noun	1.369423 sec.
de_facto	1.184568 sec.
historically	1.186245 sec.
ideology	1.343931 sec.
not_necessarily	0.707158 sec.
best_interests	0.867831 sec.
perfectly	1.378128 sec.
audience	1.277997 sec.
prestige	1.072158 sec.
either	1.113617 sec.
threat	1.415296 sec.
dream	1.507490 sec.
genius	1.174393 sec.
status_symbol	0.586222 sec.
1800 ego networks processed
above	1.396083 sec.
even	1.073855 sec.
self_interest	0.659583 sec.
actions	1.191630 sec.
phrase	1.251517 sec.
contempt	1.327365 sec.
term	1.142052 sec.
hardly	1.106717 sec.
god	1.659412 sec.
profession	1.483774 sec.
position	1.217210 sec.
humanely	0.545573 sec.
charm	1.229683 sec.
challenge	1.953760 sec.
society	1.212192 sec.
basically	1.194382 sec.
viewed_as	0.658078 sec.
but_rather	0.610058 sec.
unnecessary	1.058564 

analogues	1.423291 sec.
types	1.162683 sec.
lie	1.324442 sec.
feed	1.257256 sec.
There_are	0.574130 sec.
reside	0.675223 sec.
fall_into	0.715608 sec.
two_categories	0.547296 sec.
may_vary	0.618257 sec.
require	0.669748 sec.
inland_seas	0.652967 sec.
tend	0.692256 sec.
categories	1.353543 sec.
clusters	1.344063 sec.
can_vary	0.652406 sec.
Slow_Zone	0.497355 sec.
weren_t	0.726737 sec.
streams	1.277172 sec.
rules_apply	0.607895 sec.
classifications	1.309557 sec.
depend_on	0.829262 sec.
elytra	0.586859 sec.
include	1.196279 sec.
find_themselves	0.673705 sec.
rely_on	0.738640 sec.
canids	0.817543 sec.
operate	0.651964 sec.
vice_versa	0.628660 sec.
prefer	0.892216 sec.
have_evolved	0.645888 sec.
patches	1.207679 sec.
arrays	1.190951 sec.
different_types	0.980557 sec.
coincide	0.577302 sec.
observations_suggest	0.673406 sec.
sizes	1.081752 sec.
interact	0.716485 sec.
angles	1.421937 sec.
typically_involve	0.583065 sec.
masses	1.176598 sec.
may_differ	0.660411 sec.
shapes	1.246483 sec.
represe

On_May	0.730031 sec.
On_January	0.645260 sec.
Between	0.982543 sec.
Throughout	1.183810 sec.
On_July	0.673621 sec.
On_April	0.631051 sec.
On_December	0.603755 sec.
On_June	0.867427 sec.
Ultimately	1.337083 sec.
On_September	0.559188 sec.
On_August	0.628942 sec.
On_February	0.622869 sec.
On_November	0.660975 sec.
On_March	0.698606 sec.
2500 ego networks processed
In_contrast	0.627631 sec.
Soon_after	1.101935 sec.
Up_until	0.554635 sec.
Beginning	1.131036 sec.
Unfortunately	1.163424 sec.
Whilst	1.183652 sec.
Through	1.149906 sec.
During_this	0.654674 sec.
Shortly_after	0.986527 sec.
Accordingly	1.201522 sec.
Thereafter	1.168085 sec.
By_contrast	0.623431 sec.
Around	1.352220 sec.
By_mid	0.574089 sec.
Prior	1.119541 sec.
Along_with	1.061728 sec.
Upon	1.100757 sec.
Having	1.947137 sec.
Already	1.294903 sec.
Overall	1.378114 sec.
From_onwards	0.503537 sec.
Shortly_thereafter	0.875180 sec.
Once_again	1.194455 sec.
Besides	1.097734 sec.
Over	2.033113 sec.
Due	0.932314 sec.
coolest_months	0.533

void	1.177201 sec.
notice	1.130635 sec.
same_thing	0.844597 sec.
somebody	1.390877 sec.
defendant	1.212199 sec.
were_arrested	0.598016 sec.
surface	1.266432 sec.
concentration_camps	1.050796 sec.
happen	1.314640 sec.
statement	1.418380 sec.
zero	1.321557 sec.
blacks	1.103034 sec.
were_forcibly	0.594808 sec.
prisons	1.193521 sec.
landlords	1.185635 sec.
camps	1.365619 sec.
were_killed	0.664217 sec.
their_homes	0.638249 sec.
landowners	1.239537 sec.
Czechs	0.687698 sec.
colonists	1.260062 sec.
prisoners	1.187337 sec.
whites	0.995906 sec.
archers	0.963103 sec.
nobles	1.021210 sec.
merchants	1.058443 sec.
lodges	1.279668 sec.
sailors	1.099943 sec.
guns	1.253248 sec.
Finns	0.699929 sec.
Poles	1.183331 sec.
Iraqis	0.837499 sec.
young_men	1.007711 sec.
Egyptians	0.803318 sec.
armies	1.383809 sec.
Russians	0.751697 sec.
political_prisoners	0.584413 sec.
settlers	1.172240 sec.
raids	1.076254 sec.
cavalry	1.488319 sec.
party_cadres	0.540257 sec.
recording_equipment	0.582108 sec.
knights	1.444549

Puccini	0.854609 sec.
Honecker	0.726909 sec.
Capra	0.746658 sec.
Reagan	0.828611 sec.
Harding	0.726317 sec.
albeit	0.622299 sec.
Xavier	0.787089 sec.
Monroe	0.969825 sec.
surprisingly	0.967300 sec.
implying_that	0.766019 sec.
Booth	1.513687 sec.
Note_that	0.751301 sec.
worse	1.385512 sec.
Gable	2.014014 sec.
mindedness	0.677509 sec.
moreover	1.236979 sec.
except	1.064146 sec.
knowing	1.732562 sec.
paying_off	0.556680 sec.
3200 ego networks processed
condition	1.192051 sec.
nor	2.272936 sec.
touch	1.903795 sec.
fit	1.355745 sec.
five	1.078363 sec.
suffer	1.402951 sec.
seven	1.166788 sec.
six	1.041153 sec.
ten	1.669560 sec.
three	1.677681 sec.
eight	1.062840 sec.
ever	1.537139 sec.
four	1.736379 sec.
nine	1.254995 sec.
sixth	1.171999 sec.
tricks	1.094770 sec.
fifty	1.507792 sec.
fifth	1.235561 sec.
top	1.897198 sec.
fourth	1.032789 sec.
twelve	1.628674 sec.
spades	1.063498 sec.
twenty	1.498908 sec.
count	1.473476 sec.
bits	1.102694 sec.
thirteen	1.358133 sec.
three_times	0.669227 sec.
ar

In [ ]:
%timeit graph.get_neighbors("the")

In [20]:
def profiling(function):
    import cProfile
    import pstats
    from io import StringIO
    pr = cProfile.Profile()
    pr.enable()

    function()

    pr.disable()
    s = StringIO()
    sortby = 'cumulative'
    ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
    ps.print_stats()
    print(s.getvalue())
    
profiling(lambda: get_ego_network("the"))

0.12823724746704102
6.9550676345825195
the	7.321452 sec.
         5400995 function calls in 7.322 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    7.322    7.322 <ipython-input-20-a774236d7ace>:17(<lambda>)
        1    0.061    0.061    7.322    7.322 <ipython-input-15-d9fde47c654f>:105(get_ego_network)
      201    0.002    0.000    6.765    0.034 <ipython-input-5-9413048172d2>:62(get_neighbors)
      201    0.257    0.001    6.724    0.033 <ipython-input-5-9413048172d2>:65(<dictcomp>)
    40401    0.157    0.000    6.202    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/csr.py:236(__getitem__)
    40401    0.130    0.000    4.847    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/csr.py:368(_get_row_slice)
    40401    0.292    0.000    4.667    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/csr.py:411(_get_submat

In [ ]:
#         5400995 function calls in 7.322 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    7.322    7.322 <ipython-input-20-a774236d7ace>:17(<lambda>)
        1    0.061    0.061    7.322    7.322 <ipython-input-15-d9fde47c654f>:105(get_ego_network)
      201    0.002    0.000    6.765    0.034 <ipython-input-5-9413048172d2>:62(get_neighbors)
      201    0.257    0.001    6.724    0.033 <ipython-input-5-9413048172d2>:65(<dictcomp>)
    40401    0.157    0.000    6.202    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/csr.py:236(__getitem__)
    40401    0.130    0.000    4.847    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/csr.py:368(_get_row_slice)
    40401    0.292    0.000    4.667    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/csr.py:411(_get_submatrix)
    40401    0.336    0.000    3.695    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:25(__init__)
    40401    0.907    0.000    2.278    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:128(check_format)
    80802    0.397    0.000    0.869    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/sputils.py:119(get_index_dtype)
    40401    0.277    0.000    0.862    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/sputils.py:265(_unpack_index)
    40401    0.307    0.000    0.660    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:1065(prune)
   121203    0.130    0.000    0.433    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/sputils.py:188(isintlike)
    40401    0.390    0.000    0.390    0.000 {built-in method scipy.sparse._sparsetools.get_csr_submatrix}
        1    0.003    0.003    0.366    0.366 /home/panchenko/anaconda3/lib/python3.6/site-packages/chinese_whispers/__init__.py:31(chinese_whispers)
      400    0.113    0.000    0.360    0.001 /home/panchenko/anaconda3/lib/python3.6/site-packages/chinese_whispers/__init__.py:63(choose_label)
   404010    0.348    0.000    0.348    0.000 {built-in method numpy.core.multiarray.array}
    40401    0.154    0.000    0.338    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/sputils.py:331(_check_boolean)
   282807    0.111    0.000    0.329    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:463(asarray)
    40402    0.058    0.000    0.304    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/bidict/_frozen.py:23(proxy)
   121203    0.110    0.000    0.303    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/sputils.py:183(isscalarlike)
    80802    0.252    0.000    0.252    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/numpy/core/getlimits.py:507(__init__)
   161604    0.103    0.000    0.233    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/base.py:193(nnz)
    80802    0.103    0.000    0.225    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/csr.py:416(process_slice)
    40401    0.216    0.000    0.216    0.000 {method '__getitem__' of 'dict' objects}
   727225    0.185    0.000    0.185    0.000 {built-in method builtins.isinstance}
    36524    0.052    0.000    0.184    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/chinese_whispers/__init__.py:9(top_weighting)
    80802    0.046    0.000    0.166    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
   121203    0.134    0.000    0.161    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:2135(isscalar)
    40401    0.056    0.000    0.158    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/sputils.py:91(to_native)
    40401    0.138    0.000    0.138    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/sputils.py:200(isshape)
   161604    0.129    0.000    0.129    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:100(getnnz)
    40401    0.117    0.000    0.126    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/base.py:77(set_shape)
    80802    0.119    0.000    0.119    0.000 {built-in method builtins.hasattr}
    80802    0.078    0.000    0.116    0.000 <frozen importlib._bootstrap>:416(parent)
    37324    0.039    0.000    0.111    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/graph.py:414(__getitem__)
   421543    0.085    0.000    0.085    0.000 {built-in method builtins.len}
   121203    0.055    0.000    0.084    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/base.py:1111(isspmatrix)
    40401    0.035    0.000    0.079    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/data.py:22(__init__)
    80802    0.068    0.000    0.068    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/_lib/_util.py:128(_prune_array)
   202005    0.060    0.000    0.060    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/base.py:100(get_shape)
    80802    0.058    0.000    0.058    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/numpy/core/getlimits.py:532(max)
    80802    0.056    0.000    0.056    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/csr.py:439(check_bounds)
      200    0.037    0.000    0.054    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/graph.py:884(add_edges_from)
    37725    0.039    0.000    0.051    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/graph.py:626(nodes)
   121203    0.045    0.000    0.045    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/csr.py:231(_swap)
    40401    0.045    0.000    0.045    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/base.py:70(__init__)
    40401    0.034    0.000    0.043    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/sputils.py:293(_check_ellipsis)
    40401    0.039    0.000    0.039    0.000 {method 'newbyteorder' of 'numpy.dtype' objects}
    37324    0.030    0.000    0.038    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/graph.py:316(adj)
    80802    0.038    0.000    0.038    0.000 {method 'rpartition' of 'str' objects}
    40801    0.034    0.000    0.034    0.000 {built-in method builtins.max}
    37324    0.027    0.000    0.034    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/coreviews.py:81(__getitem__)
    40401    0.023    0.000    0.032    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/scipy/sparse/sputils.py:227(isdense)
    80804    0.030    0.000    0.030    0.000 {built-in method builtins.getattr}
      200    0.018    0.000    0.027    0.000 <ipython-input-15-d9fde47c654f>:128(<listcomp>)
    40401    0.025    0.000    0.025    0.000 {method 'indices' of 'slice' objects}
   139154    0.019    0.000    0.019    0.000 {method 'lower' of 'str' objects}
    53653    0.018    0.000    0.018    0.000 {method 'get' of 'dict' objects}
    74648    0.015    0.000    0.015    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/coreviews.py:45(__init__)
    37725    0.012    0.000    0.012    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/reportviews.py:167(__init__)
    34459    0.011    0.000    0.011    0.000 {method 'update' of 'dict' objects}
      200    0.011    0.000    0.011    0.000 {built-in method builtins.sorted}
    36524    0.011    0.000    0.011    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/coreviews.py:54(__getitem__)
    37725    0.009    0.000    0.009    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/reportviews.py:177(__getitem__)
    40000    0.009    0.000    0.009    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/graph.py:383(__contains__)
    25765    0.006    0.000    0.006    0.000 {method 'startswith' of 'str' objects}
    25765    0.006    0.000    0.006    0.000 {method 'endswith' of 'str' objects}
      200    0.005    0.000    0.005    0.000 <ipython-input-15-d9fde47c654f>:130(<listcomp>)
        2    0.000    0.000    0.001    0.000 /home/panchenko/anaconda3/lib/python3.6/random.py:260(shuffle)
        2    0.001    0.000    0.001    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/graph.py:485(add_nodes_from)
      398    0.000    0.000    0.001    0.000 /home/panchenko/anaconda3/lib/python3.6/random.py:222(_randbelow)
        3    0.000    0.000    0.000    0.000 {built-in method builtins.print}
      400    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/coreviews.py:48(__len__)
        6    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/ipykernel/iostream.py:342(write)
      400    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/coreviews.py:51(__iter__)
        9    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/ipykernel/iostream.py:180(schedule)
      401    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}
      583    0.000    0.000    0.000    0.000 {method 'getrandbits' of '_random.Random' objects}
      404    0.000    0.000    0.000    0.000 {built-in method builtins.iter}
        1    0.000    0.000    0.000    0.000 <ipython-input-15-d9fde47c654f>:114(<listcomp>)
        9    0.000    0.000    0.000    0.000 {built-in method posix.urandom}
        9    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/threading.py:1104(is_alive)
        6    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/ipykernel/iostream.py:297(_schedule_flush)
      398    0.000    0.000    0.000    0.000 {method 'bit_length' of 'int' objects}
      200    0.000    0.000    0.000    0.000 {method 'copy' of 'dict' objects}
        9    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/threading.py:1062(_wait_for_tstate_lock)
        1    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/graph.py:268(__init__)
        6    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/ipykernel/iostream.py:284(_is_master_process)
        9    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/ipykernel/iostream.py:87(_event_pipe)
        9    0.000    0.000    0.000    0.000 {method 'acquire' of '_thread.lock' objects}
        1    0.000    0.000    0.000    0.000 {method 'format' of 'str' objects}
        4    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/graph.py:365(__iter__)
        4    0.000    0.000    0.000    0.000 {built-in method time.time}
        9    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/threading.py:506(is_set)
        6    0.000    0.000    0.000    0.000 {built-in method posix.getpid}
        2    0.000    0.000    0.000    0.000 /home/panchenko/anaconda3/lib/python3.6/site-packages/networkx/classes/graph.py:397(__len__)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}



In [ ]:

1
​
